In [6]:
import pandas as pd
import pyodbc
import sqlite3

In [8]:
DB = {'servername': 'DESKTOP-I3QFF0F\SQLEXPRESS',
        'database': 'DWH'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'SQL Server' : file not found (0) (SQLDriverConnect)")